In [22]:
import sys
import os
sys.path.append(os.getcwd() + '/../')

from dataset_utils import *
from main_nusc import data_path
import pandas as pd
from ast import literal_eval

In [23]:
save_path = '../runs'
#run = 'indigo-boat'
#run = 'allegro-skillset'
run = 'violent-sack'
#run = 'relaxed-notch'
filename = 'products_lane_changes_overlapping.csv'

results_file = os.path.join(save_path, run, 'results', filename)

df = pd.read_csv(results_file)

In [24]:
accs = []
T_actions = np.zeros(len(actions))
F_actions = np.zeros(len(actions))
for idx in range(len(df)):
    item = df.iloc[idx]
    past_len = item['past_len']

    gt = literal_eval(item['gt_seq'])

    pred = literal_eval(item['prediction'])

    mask = [gt[i] == pred[i] for i in range(past_len, len(pred))]
    total = len(mask)
    correct = mask.count(True)
    acc = correct/total
    accs.append(acc)

    for i,a in enumerate(gt[past_len:len(pred)]):
        act_idx = actions_dict[a]
        if mask[i]:
            T_actions[act_idx] += 1
        else:
            F_actions[act_idx] += 1

total_actions = T_actions + F_actions
mean_actions = T_actions/total_actions

classes_acc = {actions[i]:mean_actions[i] for i in range(len(actions))}
print(classes_acc)
print(np.mean(np.array(accs)))

{'stop': 0.8276353276353277, 'back': nan, 'drive straight': 0.5129198966408268, 'accelerate': 0.5603089720736779, 'decelerate': 0.676739414288379, 'turn left': 0.6527272727272727, 'turn right': 0.5890313390313391, 'uturn': nan, 'change lane left': 0.6531393568147014, 'change lane right': 0.7368421052631579, 'overtake': 0.911522633744856, 'END': 0.7215189873417721, None: 0.8536585365853658}
0.6487214768948515


/home/muyang/miniconda3/envs/nuscenes/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


In [25]:
print(T_actions)
print(F_actions)
print(T_actions + F_actions)

[1162.    0. 2382. 1886. 3628. 1077.  827.    0. 2559. 3486.  443.  171.
  105.]
[ 242.    0. 2262. 1480. 1733.  573.  577.    0. 1359. 1245.   43.   66.
   18.]
[1404.    0. 4644. 3366. 5361. 1650. 1404.    0. 3918. 4731.  486.  237.
  123.]


NameError: name 'NuScenesSimple' is not defined

In [27]:
nusc_data = NuScenesSimple(data_path, [str(i) for i in range(2454)], mode='test', obs_perc=1)

lane_change_idxs = []
for i,n in enumerate(nusc_data):
    action_list = n['actions'].int().tolist() 
    if 8 in action_list or 9 in action_list:
        lane_change_idxs.append(i)

print(len(lane_change_idxs)/3)
    
split_dict = {'train':lane_change_idxs, 'test':lane_change_idxs}
with open('../custom_splits/lane_changes.json', 'w') as f:
    json.dump(split_dict, f)

Exception ignored in: <function BaseGeometry.__del__ at 0x7f78b6eb9cb0>
Traceback (most recent call last):
  File "/home/muyang/miniconda3/envs/nuscenes/lib/python3.7/site-packages/shapely/geometry/base.py", line 209, in __del__
    self._empty(val=None)
  File "/home/muyang/miniconda3/envs/nuscenes/lib/python3.7/site-packages/shapely/geometry/base.py", line 194, in _empty
    self._lgeos.GEOSGeom_destroy(self.__geom__)
KeyboardInterrupt: 


In [ ]:
traj = 12

render_trajectory('test_traj.gif', nusc_data[traj])